In [10]:
%pip install accelerate
%pip install transformers 
%pip install datasets
%pip install sacrebleu evaluate
%pip install torch torchvision --index-url https://download.pytorch.org/whl/cu118
%pip install transformers[sentencepiece]


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://download.pytorch.org/whl/cu118
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [11]:
from transformers import pipeline, AutoTokenizer, MT5Model, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq, AutoModelForSeq2SeqLM
from datasets import Dataset
from torch import cuda

import pandas as pd
import numpy as np
import evaluate
import csv
import os

In [12]:
def get_raw_data(path) -> None:
    """
    Get the raw data for all the books.

    Returns:
        None
    """
    raw_data = pd.read_csv(path)
    return raw_data

def prepare_data(raw_data: pd.DataFrame) -> pd.DataFrame:
    # remove the "Mefaresh" column
    raw_data.drop(columns=["Mefaresh"], inplace=True)
    # remove rows with empty values
    raw_data.dropna(inplace=True)
    # renaming the columns for mT5
    raw_data.columns = ['input_text', 'target_text']
    return raw_data


In [13]:
df = get_raw_data('./combined.csv')
df = prepare_data(df)
len(df)

343027

In [14]:
!nvidia-smi

Sun Mar 24 15:02:43 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 546.29                 Driver Version: 546.29       CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4070      WDDM  | 00000000:01:00.0  On |                  N/A |
|  0%   42C    P8              10W / 215W |    708MiB / 12282MiB |     12%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [15]:
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)
model_name = 'google/mt5-small'
tokenizer = AutoTokenizer.from_pretrained(model_name, return_tensors="pt", to_device=device)

cuda


C:\Users\almog\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\convert_slow_tokenizer.py:550: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [16]:
max_length = 40
dataset = Dataset.from_pandas(df)


def preprocess_function(examples):
    inputs = [i for i in examples["input_text"]]
    targets = [i for i in examples["target_text"]]
    model_inputs = tokenizer(
        inputs, text_target=targets, max_length=max_length, truncation=True
    )
    return model_inputs


tokenized_datasets = dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=dataset.column_names,
)

Map: 100%|██████████| 343027/343027 [00:35<00:00, 9601.79 examples/s] 


In [17]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [18]:
batch = data_collator([tokenized_datasets[i] for i in range(1, 3)])
batch.keys()

dict_keys(['input_ids', 'attention_mask', 'labels', 'decoder_input_ids'])

In [19]:
metric = evaluate.load("sacrebleu")

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    # In case the model returns more than the prediction logits
    if isinstance(preds, tuple):
        preds = preds[0]

    preds = np.where(preds != -100, preds, tokenizer.pad_token_id)

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100s in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["score"]}

In [20]:
tokenized_datasets

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 343027
})

In [25]:
args = Seq2SeqTrainingArguments(
    f"{model_name}",
    evaluation_strategy="no",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=False,
)

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets,
    eval_dataset=tokenized_datasets,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

C:\Users\almog\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\accelerate\accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [36]:
#trainer.evaluate(max_length=max_length)

In [26]:
trainer.train()

  2%|▏         | 501/32160 [01:45<1:51:09,  4.75it/s]

{'loss': 9.567, 'grad_norm': 2.8611743450164795, 'learning_rate': 1.968905472636816e-05, 'epoch': 0.05}


  3%|▎         | 1001/32160 [03:30<1:47:05,  4.85it/s]

{'loss': 5.1, 'grad_norm': 2.200582504272461, 'learning_rate': 1.937810945273632e-05, 'epoch': 0.09}


  5%|▍         | 1501/32160 [05:14<1:45:31,  4.84it/s]

...

100%|█████████▉| 32001/32160 [2:02:06<00:37,  4.27it/s]

{'loss': 3.7701, 'grad_norm': 1.2938390970230103, 'learning_rate': 9.950248756218906e-08, 'epoch': 2.99}


100%|██████████| 32160/32160 [2:02:47<00:00,  4.37it/s]

{'train_runtime': 7367.2769, 'train_samples_per_second': 139.683, 'train_steps_per_second': 4.365, 'train_loss': 4.038430110494889, 'epoch': 3.0}


TrainOutput(global_step=32160, training_loss=4.038430110494889, metrics={'train_runtime': 7367.2769, 'train_samples_per_second': 139.683, 'train_steps_per_second': 4.365, 'train_loss': 4.038430110494889, 'epoch': 3.0})

In [ ]:
trainer.evaluate(max_length=max_length)

In [27]:
model.save_pretrained("model-trained")

In [35]:
text = "בראשית ברא אלוהים את השמים ואת הארץ"
inputs = tokenizer(text, return_tensors='pt', ).to(device)
summary_ids = model.generate(**inputs, max_length=200, min_length=100, length_penalty=2.0, num_beams=4, early_stopping=True)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print(summary)

וחושך. וכן הוא חלק מן העולם, והוא אחד מן הארץ, והוא אחד מן הארץ, וכן הוא חלק מן הארץ, והוא השם הוא חלק מן העניין, וכן יהיה חלק אחד מן הדבר, וכן יראה האיש אחד, וזהו גם שם, וגם משם דכתיב דכתיב
